<a href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/TG211_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install pyCERR

In [ ]:
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

### Define data paths

In [ ]:
import os

dataDir = r'path/to/data/dir'
scan1File = os.path.join(dataDir,'PT_1','scan_2_CT.2962012.173334.2795389.7388301.nii')
mask1File = os.path.join(dataDir,'PT_1','mask_GTV_CT.2962012.173334.2795389.7388701.nii')
scan2File = os.path.join(dataDir,'PT_2','scan_3_CT.2962012.173433.677915.4034666.nii')
mask2File = os.path.join(dataDir,'PT_2','mask_GTV_CT.2962012.173433.677915.4034539.nii')


### Access imaging metadata

In [ ]:
from cerr import plan_container as pc

planC = pc.loadNiiScan(scan1File,
                       imageType='PT SCAN',
                       direction='',
                       initplanC='')
planC = pc.loadNiiScan(scan2File,
                       imageType='PT SCAN',
                       direction='',
                       initplanC=planC)
planC = pc.loadNiiStructure(mask1File, 0, planC)
planC = pc.loadNiiStructure(mask2File, 1, planC)

### Visualization

In [ ]:
from cerr import viewer as vwr

viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
            vwr.showNapari(planC, [0], [0,2], [], {}, '2d')


In [ ]:
from cerr.contour import rasterseg as rs

mask1 = rs.getStrMask(0, planC)
mask2 = rs.getStrMask(1, planC)

In [ ]:
from cerr.dataclasses import structure as cerrStr

planC = cerrStr.copyToScan(1, 0, planC)


In [ ]:
mask3 = rs.getStrMask(2, planC)
print(mask1.shape)
print(mask2.shape)
print(mask3.shape)

### Level 1 metrics

In [ ]:
from cerr.radiomics import shape as cerrShp
from cerr.radiomics import first_order as cerrFirstOrd

x1,y1,z1 = planC.scan[0].getScanXYZVals()
x2,y2,z2 = planC.scan[0].getScanXYZVals()

#Volume
rowColSlcOri1 = planC.scan[0].getScanOrientation()
shpFeats1 = cerrShp.calcShapeFeatures(mask1, x1, y1, z1, rowColSlcOri1)
shpFeats2 = cerrShp.calcShapeFeatures(mask3, x2, y2, z2, rowColSlcOri1)

#Mean uptake value, #Maximum uptake value
firstOrdFeats1 = cerrFirstOrd.stats(planC, 0)
firstOrdFeats2 = cerrFirstOrd.stats(planC, 2)

#Centre of mass
xCenter1, yCenter1, zCenter1 = cerrStr.calcIsocenter(0, planC)
xCenter2, yCenter2, zCenter2 = cerrStr.calcIsocenter(2, planC)


In [ ]:
# Volume
print('Volume =', shpFeats1['volume'], shpFeats2['volume'])

# Mean uptake value
print('Mean =', firstOrdFeats1['mean'], firstOrdFeats2['mean'])

# Max uptake value
print('Max =', firstOrdFeats1['max'], firstOrdFeats2['max'])

# Centre of mass
print('Center of mass =', xCenter1, yCenter1, zCenter1)
print('Center of mass =', xCenter2, yCenter2, zCenter2)


### Level 2 metrics

In [ ]:
from surface_distance import compute_surface_distances, compute_dice_coefficient, compute_robust_hausdorff
import numpy as np

scanNum = 0
spacing_mm = planC.scan[scanNum].getScanSpacing() * 10
surf_dists = compute_surface_distances(mask1, mask3, spacing_mm)

# DICE similarity
dsc = compute_dice_coefficient(mask1, mask3)
print(dsc)

# Hausdorff distance
hd95 = compute_robust_hausdorff(surf_dists, 95)
print('HD95 =', hd95)

# Sensitivity (S)
S = np.sum(mask1 & mask3) / np.sum(mask1)
print('S =', S)

# Positive Prediction Value (PPV)
PPV = np.sum(mask1 & mask3) / np.sum(mask3)
print('PPV =', PPV)

# Delineation Uncertainty Volume (DUV)
DUV = np.sum(mask1 | mask3) - np.sum(mask1 & mask3)
print('DUV =', DUV)

In [ ]:
print(np.sum(mask1))
print(np.sum(mask3))